In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import 

In [21]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/amaliejohansenvik/nltk_data...


True

In [14]:
df = pd.read_csv('../data/train.txt', sep=";", names=['text', 'label'])
df

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [32]:
sid = SentimentIntensityAnalyzer()
sid.polarity_scores("I am happy")

{'neg': 0.0, 'neu': 0.213, 'pos': 0.787, 'compound': 0.5719}

,text,label,Sentiment_Score
0,i didnt feel humiliated,sadness,0.504
1,i can go from feeling so hopeless to so damned...,sadness,0.271
2,im grabbing a minute to post i feel greedy wrong,anger,0.000
3,i am ever feeling nostalgic about the fireplac...,love,0.091
4,i am feeling grouchy,anger,0.278
...,...,...,...
15995,i just had a very brief time in the beanbag an...,sadness,0.112
15996,i am now turning and i feel pathetic that i am...,sadness,0.000
15997,i feel strong and good overall,joy,0.674
15998,i feel like this was such a rude comment and i...,anger,0.333


In [67]:
#3. FUNKSJON SOM LAGRER NYE FEATURES I DATASETTET VÅRT
from functools import partial
def add_features(input_df: pd.DataFrame):
    df = input_df.copy()
    def get_sentiment_score(text, key):
        return sid.polarity_scores(text)[key]
    df["Sentiment_Score_Pos"] = df["text"].apply(partial(get_sentiment_score, key="pos"))
    df["Sentiment_Score_Neu"] = df["text"].apply(partial(get_sentiment_score, key="neu"))
    df["Sentiment_Score_Neg"] = df["text"].apply(partial(get_sentiment_score, key="neg"))
    df["Sentiment_Score_Comp"] = df["text"].apply(partial(get_sentiment_score, key="compound"))
    return df
df = add_features(df)

In [41]:
from sklearn.neighbors import KNeighborsClassifier

In [82]:
knn = KNeighborsClassifier(algorithm="brute")

In [76]:
#4.TRENE DATASETTET
from sklearn.model_selection import train_test_split

#Splitter datasettet inn i en treningsdel (80%) og en testdel (20%)
train = df
test = train # add_features(pd.read_csv('../data/test.txt', sep=";", names=['text', 'label']))
knn.fit(train[["Sentiment_Score_Pos", "Sentiment_Score_Neu", "Sentiment_Score_Neg", "Sentiment_Score_Comp"]], train["label"])

KNeighborsClassifier()

In [77]:
#5. TESTE TESTDATASET
predictions = knn.predict(test[["Sentiment_Score_Pos", "Sentiment_Score_Neu", "Sentiment_Score_Neg", "Sentiment_Score_Comp"]])

In [78]:
actuals = test["label"].to_numpy()
actuals

array(['sadness', 'sadness', 'anger', ..., 'joy', 'anger', 'sadness'],
      dtype=object)

In [79]:
# 6. EVALUERE RESULTATET
def calcualte_score(predictions, actuals):
    correct_count = 0
    total = len(actuals)

    for i in range(total):
        if predictions[i] == actuals[i]:
            correct_count += 1

    return correct_count / total

In [80]:
calcualte_score(predictions,actuals)

0.6031875

In [58]:
1/6

0.16666666666666666

In [81]:
from sklearn.metrics import accuracy_score
accuracy_score(predictions, actuals)

0.6031875

In [63]:
knn.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}